In [1]:
import pandas as pd
from pydicom import dcmread
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from collections import Counter
import numpy as np
import random
import os
from matplotlib import pyplot as plt
from skimage.transform import rescale

In [2]:
all_dicoms = pd.read_csv("/home/cjmorris/dicom_practice/pt_ids/all_dicom_headers.csv")
all_dicoms = all_dicoms.dropna(subset='SeriesDescription')
svp_pids = list(pd.read_excel("/home/cjmorris/dicom_practice/pt_ids/95 SVP ground truth - HX.xlsx")['ip_patient_id'])
sa_df = all_dicoms[all_dicoms['SeriesDescription'].str.lower().str.contains('sa|short')]
sa_df = sa_df[~sa_df['SeriesDescription'].str.lower().str.contains('sag|montage|valsalva|scout|flow_quant|cemra|mpr|shortt1|delay|t2starmap|trufi_loc|loc')] #trufi_loc, loc scans have no time dimension, but are otherwise good I think
series_to_remove = ['tfl_loc_short-axis_iPAT','haste_sag','scout sa','scout SA','Cine SA montage','Delay_wideband SA_MAG','Delay_wideband SA_PSIR','MPR Ao sinuses of valsalva','MPR - sinuses of valsalva','T1Map_ShortT1_post1_19by60_MOCO','T1Map_ShortT1_post4_MOCO','T1Map_ShortT1_post3_27by60_MOCO','MPR LSA','MPR RSA','Thin mip rao gated sa_pf','Argus - EF SA Rpw','ShortIR_T1_T2map_bw1002_TR2.55_FA6_2','MPR Rt Basal','MPR gated SA','t2_tse_short axis','trufi2d_Real time SA_T-PAT3+']
# bh_cine_sa_192i_25ph_2beat_1011v4_Normalized - seem to be unable to view dicoms with slicer? 
sa_df = sa_df[~sa_df['SeriesDescription'].isin(series_to_remove)]
sa_df = sa_df[sa_df['num_frames'] >= 5]
sa_svp_df = sa_df[sa_df['PatientID'].isin(svp_pids)]
sa_svp_df['label'] = 1
sa_normal_df = sa_df[~sa_df['PatientID'].isin(svp_pids)]
sa_normal_df['label'] = 0
svp_pids = list(sa_svp_df['PatientID'].unique())
normal_pids = list(sa_normal_df['PatientID'].unique())
train_svp_pids = random.sample(svp_pids,int(np.round(len(svp_pids)*0.8)))
test_svp_pids = list(set(svp_pids) - set(train_svp_pids))
train_normal_pids = random.sample(normal_pids,int(np.round(len(normal_pids)*0.8)))
test_normal_pids = list(set(normal_pids) - set(train_normal_pids))

/tmp/ipykernel_3653129/1912044137.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa_svp_df['label'] = 1
/tmp/ipykernel_3653129/1912044137.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa_normal_df['label'] = 0


In [3]:
svp_test_csv = pd.read_csv('data_paths/svp_test.csv',header=None)
all_files = list(svp_test_csv[0])
all_dirs = list(set([os.path.dirname(file) for file in all_files]))
svp_test_df = sa_svp_df[sa_svp_df['sequence_path'].isin(all_dirs)]

normal_test_csv = pd.read_csv('data_paths/normal_test.csv',header=None)
all_files = list(normal_test_csv[0])
all_dirs = list(set([os.path.dirname(file) for file in all_files]))
normal_test_df = sa_normal_df[sa_normal_df['sequence_path'].isin(all_dirs)]

In [ ]:
med_loc_svp_test_df = pd.DataFrame(columns = [0,1]) 
for pid in svp_test_df['PatientID'].unique():
    pdf = svp_test_df[svp_test_df['PatientID'] == pid]
    for series in pdf['SeriesDescription'].unique():
        sdf = pdf[pdf['SeriesDescription'] == series]
        for date in sdf['Date'].unique():
            ddf = sdf[sdf['Date'] == date]
            all_times = dict()
            all_loc = dict()
            for dir in ddf['sequence_path'].unique():
                all_dcm_filenames = os.listdir(dir)
                dcm = dcmread(os.path.join(dir,all_dcm_filenames[0]))
                loc = float(dcm.get((0x0021,0x1041),'Unknown').value)
                all_loc[loc] = dir
            all_loc_values = list(all_loc.keys())
            upper_quartile = np.percentile(all_loc_values,75)
            lower_loc_values = [x for x in all_loc_values if x < upper_quartile]
            for val in lower_loc_values:
                dir = all_loc[val]
                for file in os.listdir(dir):
                    med_loc_svp_test_df.loc[len(med_loc_svp_test_df)] = [os.path.join(dir,file),1]
            # med_loc = np.median(all_loc_values)
            # med_loc = min(all_loc_values, key=lambda x: abs(x - med_loc))
            # test_dir = all_loc[med_loc]
            # for file in os.listdir(test_dir):
            #     dcm = dcmread(os.path.join(test_dir,file))

                    # for filename in all_dcm_filenames:
                    #     dcm = dcmread(os.path.join(dir,filename))
                    #     time = dcm.TriggerTime
                    #     all_times[time] = os.path.join(dir,filename)
                    #     print(all_times)

med_loc_normal_test_df = pd.DataFrame(columns = [0,1])
for pid in normal_test_df['PatientID'].unique():
    pdf = normal_test_df[normal_test_df['PatientID'] == pid]
    for series in pdf['SeriesDescription'].unique():
        sdf = pdf[pdf['SeriesDescription'] == series]
        for date in sdf['Date'].unique():
            ddf = sdf[sdf['Date'] == date]
            all_times = dict()
            all_loc = dict()
            for dir in ddf['sequence_path'].unique():
                all_dcm_filenames = os.listdir(dir)
                dcm = dcmread(os.path.join(dir,all_dcm_filenames[0]))
                loc = float(dcm.get((0x0020,0x1041),'Unknown').value)
                all_loc[loc] = dir
            all_loc_values = list(all_loc.keys())
            bottom_quartile = np.percentile(all_loc_values,25)
            upper_loc_values = [x for x in all_loc_values if x > bottom_quartile]
            for val in upper_loc_values:
                dir = all_loc[val]
                for file in os.listdir(dir):
                    med_loc_normal_test_df.loc[len(med_loc_normal_test_df)] = [os.path.join(dir,file),0]


In [5]:
#test new test dataset 
import torch
import torch.nn as nn
from torchvision import models
from CustomImageDataset import CustomImageDataset
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, WeightedRandomSampler
from train_and_test_loop import train_loop, test_loop
import numpy as np
import pandas as pd
import time
import wandb
upper_test_df = pd.concat([med_loc_svp_test_df,med_loc_normal_test_df])
model_path = 'resnet_weights_longrun1.pth'
model = models.resnet18(pretrained = False)
model.fc = nn.Linear(model.fc.in_features, 1,bias=True) #***this model isn't built properly***
model.load_state_dict(torch.load(model_path))
criterion = nn.BCEWithLogitsLoss()
test_dataloader = DataLoader(CustomImageDataset(upper_test_df,224,False,file_is_df=True),batch_size=100)

/home/cjmorris/miniconda3/envs/deep_learning/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cjmorris/miniconda3/envs/deep_learning/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
test_loop(test_dataloader,model,criterion,device='cpu')

Test Error: 
   Accuracy: 0.850
   recall: 0.559
   specificity: 0.943
   precision: 0.758
   AUC: 0.893
   Avg loss: 1.161674 



Error: You must call wandb.init() before wandb.log()

In [ ]:
len(sa_normal_df)*30

80730